# ゼロから作るDeep Learning
# 4章 ニューラルネットワークの学習

## 4.1 データから学習する
#### 4.1.1 データ駆動

## 4.2 損失関数
#### 4.2.1 2乗和誤差
損失関数の中で最も有名なものは次の2乗和誤差(Mean Squared Error)である。
（平均2乗誤差の方が一般的なイメージか？）
\begin{equation}
E = \frac{1}{2} \sum_{k} (y_{k} - t_{k})^{2}
\end{equation}
ここで、$y_{k}$はニューラルネットワークの出力、$t_{k}$は教師データを表す。  
$y_{k}$の分布に正規分布を仮定した場合の対数尤度関数の定数項を除いたものである。

In [1]:
# 必要なモジュールを読み込む
import numpy as np

In [2]:
# 平均二乗誤差の実装
def mean_squared_error(y, t):
    
    return 0.5 * np.sum((y - t)**2)

In [3]:
# 実際に試してみる
print('----- Mean Squared Error -----')
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

# 2の確率が最も悪い場合
y1 = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
print('Example 1:', mean_squared_error(y1, t))

# 2の確率が最も高い場合
y2 = np.array([0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0])
print('Example 2:', mean_squared_error(y2, t))

----- Mean Squared Error -----
Example 1: 0.0975
Example 2: 0.5975


#### 4.2.2 交差エントロピー誤差
交差エントロピー誤差(Cross Entropy Error)もよく用いられる損失関数で、次式で表される。
\begin{equation}
E = -\sum_{k} t_{k} \log y_{k}
\end{equation}
これは多項ロジスティック回帰で回帰係数を求める際の対数尤度関数である。
(Bishop, pp.235を参照)

In [4]:
# 交差エントロピー誤差の実装
def cross_entropy_error(y, t):
    # yが0だった場合でも計算を進められるようにするため
    delta = 1e-7

    return -np.sum(t * np.log(y + delta))

In [5]:
# 実際に試してみる
print('----- Cross Entropy Error -----')
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

y1 = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
print('Example 1:', cross_entropy_error(y1, t))

y2 = np.array([0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0])
print('Example 2:', cross_entropy_error(y2, t))

----- Cross Entropy Error -----
Example 1: 0.510825457099
Example 2: 2.30258409299


#### 4.2.3 ミニバッチ学習
先程の交差エントロピー誤差はデータが1つの場合のものである。
これを$N$個のデータに適用できるように拡張すると次のようになる。
\begin{equation}
E = - \frac{1}{N} \sum_{n} \sum_{k} t_{nk} \log y_{nk}
\end{equation}

ミニバッチ学習とは数百万、数千万という膨大なデータの中から一部を抜き出し、その抜き出したデータを使って学習を行うこと。
ミニバッチ学習のために訓練データの中から指定された個数のデータをランダムに選び出すコードを書く。

In [6]:
# MNISTデータを読み込む
import sys
import os
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000, 10)


In [7]:
# 実際に訓練データを抜き出すコード
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

#### 4.2.4 ［バッチ対応版］交差エントロピー誤差の実装
データが1つの場合と、データがバッチとしてまとめられて入力される場合の両方のケースに対応できるようにする。

In [8]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = t.reshape(1, y.size)
    
    batch_size = y.shape[0]
    
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

In [9]:
# reshapeメソッドを知らなかったので、動作検証してみる
test = np.array([0.1, 2, 0.5, 9])
print('test =', test)
print('test.reshape(1, test.size) =', test.reshape(1, test.size))

test = [ 0.1  2.   0.5  9. ]
test.reshape(1, test.size) = [[ 0.1  2.   0.5  9. ]]


In [10]:
# 複雑な部分の動作を確認する
y = np.array([[0.5, 1, 0.1],
              [0.2, 2, 7],
              [10, 3, 0]])
print('y =', y)

t = np.array([2, 0, 1])
print('t =', t)

print('y[np.arange(3), t] =', y[np.arange(3), t])
print('y[:, t] =', y[:, t])

y = [[  0.5   1.    0.1]
 [  0.2   2.    7. ]
 [ 10.    3.    0. ]]
t = [2 0 1]
y[np.arange(3), t] = [ 0.1  0.2  3. ]
y[:, t] = [[  0.1   0.5   1. ]
 [  7.    0.2   2. ]
 [  0.   10.    3. ]]


## 4.3 数値微分
#### 4.3.1 微分

In [11]:
def numerical_diff(f, x):

    h = 1e-4

    return (f(x + h) - f(x - h)) / (2 * h)

#### 4.3.2 数値微分の例
上記の数値微分で次の関数を試してみる。
\begin{equation}
y = 0.01 x^{2} + 0.1 x
\end{equation}

In [12]:
def f1(x):

    return 0.01 * x**2 + 0.1 * x

# 数値微分を試す
print('numerical_diff(f1,  5) =', numerical_diff(f1, 5))
print('numerical_diff(f1, 10) =', numerical_diff(f1, 10))

numerical_diff(f1,  5) = 0.1999999999990898
numerical_diff(f1, 10) = 0.2999999999986347


#### 4.3.3 偏微分
2変数関数
\begin{equation}
f(x_{0}, x_{1}) = x_{0}^{2} + x_{1}^{2}
\end{equation}
について、  
(1)$x_{0}=3$, $x_{1}=4$のときの$x_{0}$に対する偏微分$\frac{\partial f}{\partial x_{0}}$を求めよ。  
(2)$x_{0}=3$, $x_{1}=4$のときの$x_{1}$に対する偏微分$\frac{\partial f}{\partial x_{1}}$を求めよ。  

In [13]:
def f2(x):
    
    return np.sum(x**2)


def f2_tmp1(x0):

    return x0 ** 2 + 4.0 ** 2


def f2_tmp2(x1):
    
    return 3.0 ** 2 + x1 ** 2

print('numerical_diff(f2_tmp1, 3.0) =', numerical_diff(f2_tmp1, 3.0))
print('numerical_diff(f2_tmp2, 4.0) =', numerical_diff(f2_tmp2, 4.0))

numerical_diff(f2_tmp1, 3.0) = 6.00000000000378
numerical_diff(f2_tmp2, 4.0) = 7.999999999999119


## 4.4 勾配
2変数関数の勾配(gradient)
\begin{equation}
\nabla f(x0, x1) = (\frac{\partial f}{\partial x_{0}}, \frac{\partial f}{\partial x_{1}})
\end{equation}
を実装する。

In [14]:
# 全微分の実装
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)

    for i, tmp in enumerate(x):
        tmp_x = x.copy()
        
        # f(x + h)の計算
        tmp_x[i] = tmp + h
        fxh1 = f(tmp_x)
        
        # f(x - h)の計算
        tmp_x[i] = tmp - h
        fxh2 = f(tmp_x)
        
        grad[i] = (fxh1 - fxh2) / (2 * h)
    
    return grad


# テスト
print('numerical_gradient(f2, np.array([3, 4])) =', numerical_gradient(f2, np.array([3.0, 4.0])))
print('numerical_gradient(f2, np.array([0, 2])) =', numerical_gradient(f2, np.array([0.0, 2.0])))
print('numerical_gradient(f2, np.array([3, 0])) =', numerical_gradient(f2, np.array([3.0, 0.0])))

numerical_gradient(f2, np.array([3, 4])) = [ 6.  8.]
numerical_gradient(f2, np.array([0, 2])) = [ 0.  4.]
numerical_gradient(f2, np.array([3, 0])) = [ 6.  0.]


In [15]:
# zeros_likeメソッドの動作確認
x = np.arange(10)
y = np.zeros_like(x)

print('x =', x)
print('y =', y)

x = [0 1 2 3 4 5 6 7 8 9]
y = [0 0 0 0 0 0 0 0 0 0]


#### 4.4.1 勾配法

In [16]:
# 勾配法を実装する
def gradient_desent(f, init_x, lr=0.01, step_num=100):
    # init_xを変更させないようにするため
    x = init_x.copy()
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    
    return x

In [17]:
# 実装した勾配法を試す
gradient_desent(
    f=f2,
    init_x=np.array([-3.0, 4.0]),
    lr=0.1,
    step_num=100
)

array([ -6.11110793e-10,   8.14814391e-10])

In [18]:
init_x = np.array([-3.0, 4.0])

# 学習率が大きすぎる例
print('学習率が大きすぎる例(lr = 10.0): ', gradient_desent(f=f2, init_x=init_x, lr=10.0, step_num=100))
print('学習率が小さすぎる例(lr = 1e-10): ', gradient_desent(f=f2, init_x=init_x, lr=1e-10, step_num=100))

学習率が大きすぎる例(lr = 10.0):  [ -2.58983747e+13  -1.29524862e+12]
学習率が小さすぎる例(lr = 1e-10):  [-2.99999994  3.99999992]


#### 4.4.2 ニューラルネットワークに対する勾配

In [19]:
import sys
import os
import numpy as np
from common.functions import (softmax, cross_entropy_error)
from common.gradient import numerical_gradient

class SimpleNet(object):
    def __init__(self):
        self.W = np.random.randn(2, 3)
    
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss

In [20]:
# SimpleNetを試す
net = SimpleNet()
print('net.W =', net.W)

x = np.array([0.6, 0.9])
p = net.predict(x)
print('p =', p)
print('np.argmax(p) =', np.argmax(p))

t = np.array([0, 0, 1])
print('net.loss(x, t) =', net.loss(x, t))

net.W = [[-0.53572577  0.65692054  0.1165997 ]
 [-0.37309075 -1.30970997 -0.51098734]]
p = [-0.65721714 -0.78458665 -0.38992879]
np.argmax(p) = 2
net.loss(x, t) = 0.891736898302


In [21]:
# なぜダミーでWを引数に設定するのか分からない
def f(W):
    return net.loss(x, t)

dW = numerical_gradient(f, net.W)
print('dW =', dW)

dW = [[ 0.18827514  0.165759   -0.35403414]
 [ 0.28241271  0.24863849 -0.53105121]]


#### 4.5.1 2層ニューラルネットワークのクラス
初めに、2層ニューラルネットワークを1つのクラスとして実装することから始める。

In [22]:
import sys
import os
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet(object):
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}

        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    
    def predict(self, x):
        W1 = self.params['W1']
        b1 = self.params['b1']
        
        W2 = self.params['W2']
        b2 = self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    def numerical_gradient(self, x, t):
        def loss_W(W):
            return self.loss(x, t)
            
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

In [23]:
# TwoLayerNetを試す
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)

print('----- 重みとバイアス -----')
print("net.params['W1'].shape =", net.params['W1'].shape)
print("net.params['b1'].shape =", net.params['b1'].shape)
print("net.params['W2'].shape =", net.params['W2'].shape)
print("net.params['b2'].shape =", net.params['b2'].shape)

----- 重みとバイアス -----
net.params['W1'].shape = (784, 100)
net.params['b1'].shape = (100,)
net.params['W2'].shape = (100, 10)
net.params['b2'].shape = (10,)


In [24]:
# 推論処理
x = np.random.rand(100, 784)
y = net.predict(x)

print('----- 推論処理 -----')
print('y =', y)

----- 推論処理 -----
y = [[ 0.10096015  0.09518637  0.09740336  0.09756574  0.10103248  0.09787467
   0.10726158  0.09685721  0.09747524  0.1083832 ]
 [ 0.10069684  0.09474417  0.09783472  0.0970222   0.1012451   0.0979781
   0.10733542  0.09710959  0.09759657  0.10843729]
 [ 0.10097832  0.09507847  0.09771713  0.097256    0.1011199   0.09760068
   0.1070632   0.09713756  0.09724943  0.10879932]
 [ 0.1011573   0.094865    0.09758215  0.09757701  0.10089678  0.09741444
   0.10746093  0.09681855  0.09745138  0.10877645]
 [ 0.10076386  0.09504501  0.09739035  0.0975692   0.10135993  0.09774502
   0.10691646  0.09686021  0.09756259  0.10878737]
 [ 0.10091879  0.0950426   0.09776218  0.09713894  0.1013519   0.09798323
   0.1071294   0.09691193  0.09749831  0.10826271]
 [ 0.10102461  0.0950317   0.09786823  0.09735454  0.10109877  0.0977426
   0.10702709  0.0969082   0.09778126  0.10816299]
 [ 0.10115677  0.09503313  0.09775602  0.09765026  0.10106137  0.09774827
   0.1070723   0.09693788  0.097

In [25]:
# 勾配の計算
x = np.random.rand(100, 784)
t = np.random.rand(100, 10)

grads = net.numerical_gradient(x, t)

print('----- 勾配の計算 -----')
print("grads['W1'] =", grads['W1'].shape)
print("grads['b1'] =", grads['b1'].shape)
print("grads['W2'] =", grads['W2'].shape)
print("grads['b2'] =", grads['b2'].shape)

----- 勾配の計算 -----
grads['W1'] = (784, 100)
grads['b1'] = (100,)
grads['W2'] = (100, 10)
grads['b2'] = (10,)


#### 4.5.2 ミニバッチ学習の実装
TwoLayerNetクラスを対象に、MNISTデータセットを使って学習を行う。

## 6時間近く回しても結果が返って来なかった、、、

In [26]:
import sys
import numpy as np
from dataset.mnist import load_mnist
# 下記のモジュールは上で実装したTwoLayerNetと同じため、ここではコメントアウトしておく
# sys.path.append('/home/ishiyama/deep_learning/deep-learning-from-scratch/ch04')
# from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

# hyperparameters
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # ミニバッチの取得
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 勾配の計算
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # パラメータの更新
    for key in network.params:
        network.params[key] -= learning_rate * grad[key]
    
    # 学習経過の記録
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

KeyboardInterrupt: 

#### 4.5.3 テストデータで評価

In [27]:
import numpy as np
from dataset.mnist import load_mnist
# from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []
train_acc_list = []
test_acc_list = []
# 1エポックあたりの繰り返し数
iter_per_epoch = max(train_size / batch_size, 1)

# ハイパーパラメータ
iters_num = 10000
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train(batch_mask)
    t_batch = t_train(batch_mask)
    
    # 勾配の計算
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # パラメータの更新
    for key in grad:
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1エポックごとに認識精度を計算
    if not (i % iter_per_epoch):
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print('train_acc, test_acc | {}, {}'.format(train_acc, test_acc))


TypeError: 'numpy.ndarray' object is not callable

## 参考文献
斎藤(2016), ゼロから作るDeep Learning, O'reilly Japan  
Bishop(2006), Pattern Recognition and Machine Learning, Springer-Verlag New York